# Houm // Challenge Data
Respondido por: Johans Peña Arias
- Primero se importan los datos desde las fuentes entregadas para el desafío, se realizan validaciones de datos duplicados y se realizan algunas transformaciones. Luego se pasa al desarrollo de las respuestas.

## Importación de datos
### 1.1. Importar datos CSV y API

In [1]:
import pandas as pd
#PROPERTIES
df_properties=pd.read_csv("properties.csv")
df_properties[df_properties.duplicated(keep=False)] #Se valida si existen duplicados

,property_id,type_house,business_type,bedrooms,bathrooms,parking_lots,services,balcony,pool,latitude,longitude,localidad,city,region,country


In [2]:
#USERS
df_users=pd.read_csv("users.csv")
df_users[df_users.duplicated(keep=False)]#Se valida si existen duplicados

,property_id,user_id,first_name,last_name,address


In [3]:
#VISITS

df_visits=pd.read_csv("visits.csv")
df_visits.begin_date=df_visits.begin_date.replace({'-03:00':''}, regex=True)
df_visits.end_date=df_visits.end_date.replace({'-03:00':''}, regex=True)
df_visits[df_visits.duplicated(keep=False)]#Se valida si existen duplicados

,scheduled_id,property_id,begin_date,end_date,type_visit,status


#### 1.2. Importar datos desde API

In [83]:
import json
import codecs
import urllib.request
import urllib.error

def Api_info(lat,long,date_hour):
    BaseURL = 'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/'

    ApiKey='YDTMZDZ74DXY5A6W2KTY3VGTY'
    UnitGroup='metric'#UnitGroup sets the units of the output - us or metric
    Location=str(lat)+","+str(long) #Location for the weather data

    Date_hour= date_hour
    ContentType="json"#JSON or CSV 
    Include="current"#values include days,hours,current,alerts
#     Elements="tempmax,tempmin,temp"

    ApiQuery=BaseURL + Location + "/"+Date_hour #basic query including location
#     if (len(StartDate)):#append the start and end date if present
#     ApiQuery+="/"+StartDate
#     if (len(EndDate)):
#         ApiQuery+="/"+EndDate

    ApiQuery+="?"#Url is completed.
    if (len(UnitGroup)):#append each parameter as necessary
        ApiQuery+="&unitGroup="+UnitGroup

    if (len(ContentType)):
        ApiQuery+="&contentType="+ContentType

    if (len(Include)):
        ApiQuery+="&include="+Include

    ApiQuery+="&key="+ApiKey
#     print(ApiQuery)
    jsonBytes = urllib.request.urlopen(ApiQuery)
    info_dic=json.load(jsonBytes)
    
    return(info_dic)

## Desafío:


1. ¿Cuántas visitas se realizaron en total?
2. ¿Cuál es el promedio de propiedades por usuario?
3. ¿Cuál era la temperatura promedio de todas las visitas que realizó el usuario con ID 2?
4. ¿Cuál es la temperatura promedio de las visitas para los días con lluvia?
5. ¿Cuál es la temperatura promedio para las visitas realizadas en la localidad de Suba?

#### 1. ¿Cuántas visitas se realizaron en total?
Respuesta: Se realizaron 139 visitas, de 425 agendadas.

In [6]:
# df_visits.type_visit.value_counts() #425 (visitas agendadas)
# df_visits.status.value_counts() #Cancelled    286 y Done         139
df_visits[df_visits['status']=="Done"].reset_index(drop=True)

,scheduled_id,property_id,begin_date,end_date,type_visit,status
0,181607,2,2022-01-20T13:00:00,2022-01-20T15:00:00,Visit,Done
1,177048,3,2022-01-14T19:30:00,2022-01-14T21:30:00,Visit,Done
2,164793,3,2022-01-10T13:00:00,2022-01-10T15:00:00,Visit,Done
3,168308,5,2022-01-07T11:00:00,2022-01-07T13:00:00,Visit,Done
4,171942,5,2022-01-09T16:00:00,2022-01-09T18:00:00,Visit,Done
...,...,...,...,...,...,...
134,205772,78,2022-02-24T18:30:00,2022-02-24T20:00:00,Visit,Done
135,205520,78,2022-02-24T18:00:00,2022-02-24T20:00:00,Visit,Done
136,208629,80,2022-02-27T16:00:00,2022-02-27T18:00:00,Visit,Done
137,208045,80,2022-02-27T16:00:00,2022-02-27T18:00:00,Visit,Done


#### 2. ¿Cuál es el promedio de propiedades por usuario?
Respuesta: El promedio de propiedades por usuario es 1.

In [7]:
df_prop_users = pd.merge(df_properties,df_users, on='property_id')
count_prop=df_prop_users.groupby(['user_id']).size().reset_index(name='Cantidad_propiedades')
count_prop.Cantidad_propiedades.mean()

1.0

#### 3. ¿Cuál era la temperatura promedio de todas las visitas que realizó el usuario con ID 2?
- Respuesta: 18.26 grados celsius
- Desarrollo: El desarrollo de la respuesta se divide en tres partes, preparación de los datos, las consultas a la API para rescatar los datos externos y creación de DF consolidado.

##### A. Se crea nuevo dataframe a partir de los 3 archivos csv.

In [5]:
df_visits_properties =pd.merge(df_visits[['scheduled_id', 'property_id', 'begin_date', 'end_date','status']],
                                df_properties[
                                    ['property_id','latitude', 'longitude','localidad', 'city', 'region', 'country']], 
                               on='property_id', how='left')#Merge entre df_visits y df_properties

df_visits_properties_user =pd.merge(df_visits_properties,df_users[['property_id','user_id']], 
                                   on='property_id', how='left')#Merge entre df_visits/df_properties y df_user

df_visits_properties_user2 =df_visits_properties_user[(df_visits_properties_user['user_id']==2) & 
                          (df_visits_properties_user['status']=='Done')].reset_index(drop=True)
                            #Se crea nuevo 'DF' y se filtra de acuerdo a status y user_id, se dejan solo las columnas con información relevante
df_visits_properties_user2

,scheduled_id,property_id,begin_date,end_date,status,latitude,longitude,localidad,city,region,country,user_id
0,187080,34,2022-01-28T11:00:00,2022-01-29T01:00:00,Done,4.618363,-74.0735,Teusaquillo,Bogotá,Región De Cundinamarca,Colombia,2
1,186857,34,2022-01-28T11:30:00,2022-01-28T14:00:00,Done,4.618363,-74.0735,Teusaquillo,Bogotá,Región De Cundinamarca,Colombia,2


##### B. Consulta API
Se crea un bucle for que itera sobre el Dataframe 'df_visits_properties_user2', para extraer la información relativa a beginDate, endDate, latitude y longitude, de las visitas realizadas por el usuario 2.

In [93]:
df_temp_visit=[]

for i in range(len(df_visits_properties_user2)):
    lat=df_visits_properties_user2.iloc[i,5] #Se obtiene información de la latitud para la visita
    long=df_visits_properties_user2.iloc[i,6] #Se obtiene información de la longitud para la visita
    beginDate=df_visits_properties_user2.iloc[i,2] #Se obtiene información sobre la hora de inicio de la visita
    endDate=df_visits_properties_user2.iloc[i,3] #Se obtiene información sobre la hora de termino de la visita
    lista_date_hour = list(pd.date_range(beginDate, endDate, 
                                         freq="H" ).strftime('%Y-%m-%dT%H:%M:%S'))#Se crea una lista con la fecha y horas que dura la visita.
    
    temp_visit=[]#Lista para guardar las temperaturas de cada hora en grados celsius.
    
    for j in lista_date_hour: #Se recorre el listado de horas y se consulta la Api por cada una, pra obtener la temperatura por hora.
        if j.split('T')[1]=='00:00:00':
            temp_visit.append(Api_info(lat,long,j)['days'][0]['temp'])
        else:
            temp_visit.append(Api_info(lat,long,j)['currentConditions']['temp'])

    df_temp_visit.append(round(sum(temp_visit)/len(temp_visit), 2))#Se calcula el promedio de temperatura por todo el periodo de la visita.

##### C. Creación DF consolidado con datos de temperatura.

In [7]:
df_visits_properties_user2['Temp_prom']=df_temp_visit #Se crea una nueva columna con la temperatura promedio por cada visita
df_p3=df_visits_properties_user2.loc[:, ['scheduled_id','user_id', 'Temp_prom']]
df_p3
# round(sum(df_p3.Temp_prom)/len(df_p3.Temp_prom), 2)
# 18.26

,scheduled_id,user_id,Temp_prom
0,187080,2,15.99
1,186857,2,20.53


#### 4. ¿Cuál es la temperatura promedio de las visitas para los días con lluvia?
- Respuesta: La temperatura promedio de cada visita con dias de lluvia, se puede revisar en el DF ´df_visits_done_rain´, el promedio de todas las visitas es 14.19 grados celsius.
- Desarrollo: El desarrollo de la respuesta se divide en tres partes, preparación de los datos, las consultas a la API para rescatar los datos externos y creación de DF consolidado.

##### A. Se crea nuevo dataframe a partir del DF 'df_visits_properties', con el filtro en la columna status, Done.

In [81]:
df_visits_done=df_visits_properties[df_visits_properties['status']=="Done"].reset_index(drop=True)
df_visits_done.head()

,scheduled_id,property_id,begin_date,end_date,status,latitude,longitude,localidad,city,region,country
0,181607,2,2022-01-20T13:00:00,2022-01-20T15:00:00,Done,4.623068,-74.074030,Teusaquillo,Bogotá,Región De Cundinamarca,Colombia
1,177048,3,2022-01-14T19:30:00,2022-01-14T21:30:00,Done,4.723909,-74.042336,Usaquen,Bogotá,Región De Cundinamarca,Colombia
2,164793,3,2022-01-10T13:00:00,2022-01-10T15:00:00,Done,4.723909,-74.042336,Usaquen,Bogotá,Región De Cundinamarca,Colombia
3,168308,5,2022-01-07T11:00:00,2022-01-07T13:00:00,Done,4.688163,-74.043300,Usaquen,Bogotá,Región De Cundinamarca,Colombia
4,171942,5,2022-01-09T16:00:00,2022-01-09T18:00:00,Done,4.688163,-74.043300,Usaquen,Bogotá,Región De Cundinamarca,Colombia


##### B. Consulta API
Se crea un bucle for que itera sobre el Dataframe 'df_visits_done', para extraer la información relativa a beginDate, latitude y longitude, de las visitas realizadas.

In [70]:
df_rain_temp=[]

for i in range(len(df_visits_done)):
    lat=df_visits_done.iloc[i,5] #Se obtiene información de la latitud para la visita
    long=df_visits_done.iloc[i,6] #Se obtiene información de la longitud para la visita
    beginDate=df_visits_done.iloc[i,2] #Se obtiene información sobre la hora de inicio de la visita y se considera solo la fecha.
    
    if 'Rain' in Api_info(lat,long,beginDate)['days'][0]['conditions']: #Encuentra la palabra Rain en conditions.
        df_rain_temp.append(Api_info(lat,long,beginDate)['days'][0]['temp']) #Devuelve la temperatura si encuentra Rain.
    else:
        df_rain_temp.append('SinLluvia')#resultado para un día sin lluvia.  

##### C. Creación DF consolidado con datos de temperatura de los días con lluvia.

In [82]:
df_visits_done['Lluvia_temp']=df_rain_temp #Se crea una nueva columna con la temperatura promedio por cada visita
df_visits_done_rain=df_visits_done[df_visits_done['Lluvia_temp']!="SinLluvia"].reset_index(drop=True)
# df_visits_done_rain.head()
round(sum(df_visits_done_rain.Lluvia_temp)/len(df_visits_done_rain.Lluvia_temp), 2) #14.19

14.19

#### 5. ¿Cuál es la temperatura promedio para las visitas realizadas en la localidad de Suba?
- Respuesta: La temperatura promedio por cada las visitas realizadas en la localidad de Suba, se encuentra detallada en el DF ´df_p5´. El promedio de todas las visitas es 18.21 grados celsius.
- Desarrollo: El desarrollo de la respuesta se divide en 3 partes, preparación de los datos, las consultas a la API para rescatar los datos externos y construcción de dataframe con datos consolidados.

##### A. Se crea nuevo dataframe usando el DF df_visits_done, filtrado por la localidad Suba, de la pregunta 4.

In [68]:
df_visits_suba=df_visits_done[df_visits_done['localidad']=='Suba'].reset_index(drop=True)
df_visits_suba.head()

,scheduled_id,property_id,begin_date,end_date,status,latitude,longitude,localidad,city,region,country
0,167764,10,2022-01-04T13:00:00,2022-01-04T15:00:00,Done,4.723203,-74.053140,Suba,Bogotá,Región De Cundinamarca,Colombia
1,176814,19,2022-01-14T15:00:00,2022-01-14T17:00:00,Done,4.767047,-74.048990,Suba,Bogotá,Región De Cundinamarca,Colombia
2,166495,19,2022-01-03T15:30:00,2022-01-03T17:30:00,Done,4.767047,-74.048990,Suba,Bogotá,Región De Cundinamarca,Colombia
3,178497,28,2022-01-17T15:30:00,2022-01-17T18:00:00,Done,4.753546,-74.093185,Suba,Bogotá,Región De Cundinamarca,Colombia
4,177764,28,2022-01-17T17:00:00,2022-01-17T19:00:00,Done,4.753546,-74.093185,Suba,Bogotá,Región De Cundinamarca,Colombia


##### B. Se crea un bucle for que itera sobre el Dataframe 'df_visits_suba', para extraer la información relativa a beginDate, endDate, latitude y longitude. Se realizan consultas a la API por cada hora de la visita y luego se determina el promedio por visita.

In [48]:
df_temp_suba=[]

for i in range(len(df_visits_suba)):
    lat=df_visits_suba.iloc[i,5] #Se obtiene información de la latitud para la visita
    long=df_visits_suba.iloc[i,6] #Se obtiene información de la longitud para la visita
    beginDate=df_visits_suba.iloc[i,2] #Se obtiene información sobre la hora de inicio de la visita
    endDate=df_visits_suba.iloc[i,3] #Se obtiene información sobre la hora de termino de la visita
    lista_date_hour_suba = list(pd.date_range(beginDate, endDate, 
                                         freq="H" ).strftime('%Y-%m-%dT%H:%M:%S'))#Se crea una lista con la fecha y horas que dura la visita.
    
    temp_visit_suba=[]#Lista para guardar las temperaturas de cada hora en grados celsius.
    
    for j in lista_date_hour_suba: #Se recorre el listado de horas y se consulta la Api por cada una, pra obtener la temperatura por hora.
        if j.split('T')[1]=='00:00:00':
            temp_visit_suba.append(Api_info(lat,long,j)['days'][0]['temp'])          
        else:
            temp_visit.append(Api_info(lat,long,j)['currentConditions']['temp'])
    
    temp_visit=[i for i in temp_visit if i]
    df_temp_suba.append(round(sum(temp_visit)/len(temp_visit), 2))#Se calcula el promedio de temperatura por todo el periodo de la visita.

##### C. Creación DF consolidado con datos de temperatura.

In [69]:
df_visits_suba['Temp_prom_suba']=df_temp_suba #Se crea una nueva columna con la temperatura promedio por cada visita
df_p5=df_visits_suba.loc[:, ['scheduled_id','localidad','Temp_prom_suba']]
df_p5.head()
# round(sum(df_p5.Temp_prom_suba)/len(df_p5.Temp_prom_suba), 2) #18.21

,scheduled_id,localidad,Temp_prom_suba
0,167764,Suba,18.65
1,176814,Suba,18.56
2,166495,Suba,18.59
3,178497,Suba,18.54
4,177764,Suba,18.39
